In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from contextlib import contextmanager
from pathlib import Path

from fastcore.basics import patch

In [ ]:
#| export
from hwpapi.actions import _Action, _Actions
from hwpapi.dataclasses import CharShape, ParaShape
from hwpapi.functions import (
    check_dll,
    dispatch,
    get_absolute_path,
    get_charshape_pset,
    get_parashape_pset,
    get_rgb_tuple,
    get_value,
    mili2unit,
    point2unit,
    set_charshape_pset,
    set_parashape_pset,
    set_pset,
    unit2mili,
    unit2point,
)

In [ ]:
#| export
class App:
    """App 클래스는 한컴오피스의 한/글 프로그램과 상호작용하기 위한 인터페이스를 제공합니다."""

    def __init__(self, api=None, is_visible=True):
        """`__init__` 함수에서는 `api` 객체를 인자로 받습니다.
        만약 `api`가 제공되지 않았을 경우, `wc.gencache.EnsureDispatch("HWPFrame.HwpObject")`를 호출하여
        기본값으로 한/글 프로그램의 COM 객체를 생성합니다. 그리고 `self.api` 속성에 이 객체를 할당합니다.
        `_Actions` 클래스의 객체를 생성하여 `self.actions` 속성에 할당하고, `self.set_visible()` 함수를 호출합니다.
        """
        if not api:
            api = dispatch("HWPFrame.HwpObject")
        self.api = api
        self.actions = _Actions(self)
        self.parameters = api.HParameterSet
        self.set_visible(is_visible)
        check_dll()
        self.api.RegisterModule("FilePathCheckDLL", "FilePathCheckerModule")

    def __str__(self):
        return f"<Hwp App: {self.get_filepath()}>"

    __repr__ = __str__

In [ ]:
#| export


@patch
def reload(app: App):
    app.api = dispatch("HWPFrame.HwpObject")
    app.set_visible()
    check_dll()
    app.api.RegisterModule("FilePathCheckDLL", "FilePathCheckerModule")

In [ ]:
#| export
@patch
def set_visible(app: App, is_visible=True, window_i=0):
    """`set_visible()` 함수는 한/글 프로그램의 창을 화면에 보이거나 숨기기 위해 호출됩니다.
    `is_visible` 인자가 `True`일 경우 창이 화면에 보이고, `False`일 경우 숨깁니다.
    `window_i` 인자는 창의 인덱스를 지정합니다."""

    app.api.XHwpWindows.Item(window_i).Visible = is_visible

In [ ]:
#| export
@patch
def get_filepath(app: App):
    """`get_filepath()` 함수는 현재 열려있는 한/글 문서의 경로를 반환합니다."""
    doc = app.api.XHwpDocuments.Active_XHwpDocument
    return doc.FullName

In [ ]:
app = App()

In [ ]:
#| export
@patch
def open(app: App, path: str):
    """`open()` 함수는 파일 경로를 인자로 받아 해당 파일을 한/글 프로그램에서 엽니다.
    `get_absolute_path()` 함수를 호출하여 절대 경로로 변환한 후, `api.Open()` 함수를 호출하여 파일을 엽니다.
    열린 파일의 경로를 반환합니다."""
    name = get_absolute_path(path)
    app.api.Open(name)
    return name

In [ ]:
app.open("test/공문양식.hwp")

'C:/Users/freed/Documents/python_projects/hwpapi/nbs/02_api/test/공문양식.hwp'

In [ ]:
#| export
@patch
def save(app: App, path=None):
    """`save()` 함수는 현재 열려있는 문서를 저장하거나 다른 이름으로 저장합니다.
    `path` 인자가 주어지지 않은 경우 현재 문서를 덮어쓰기로 저장하고, 저장된 파일의 경로를 반환합니다.
    `path` 인자가 주어진 경우, `Path` 모듈을 이용하여 파일 확장자를 추출한 후, 해당 확장자에 맞게 문서를 저장합니다.
    저장된 파일의 경로를 반환합니다."""

    if not path:
        app.api.Save()
        return app.get_filepath()
    name = get_absolute_path(path)
    extension = Path(name).suffix
    format_ = {
        ".hwp": "HWP",
        ".pdf": "PDF",
        ".hwpx": "HWPML2X",
        ".png": "PNG",
    }.get(extension)

    app.api.SaveAs(name, format_)
    return name

In [ ]:
#| export
@patch
def close(app: App):
    """`close()` 함수는 현재 열려있는 문서를 닫습니다."""
    app.api.Run("FileClose")

In [ ]:
app.close()

In [ ]:
#| export
@patch
def quit(app: App):
    """`quit()` 함수는 한/글 프로그램을 종료합니다."""
    app.api.Run("FileQuit")

In [ ]:
#| export
@patch
def create_action(app: App, action_key: str):
    """`create_action()` 함수는 `_Action` 클래스의 객체를 생성하여 반환합니다."""
    return _Action(app, action_key)

In [ ]:
#| export
@patch
def create_parameterset(app: App, action_key: str):
    """`create_parameterset()` 함수는 특정 액션의 파라미터셋을 반환합니다.
    `_action_info` 딕셔너리에서 액션에 대한 정보를 찾아서 파라미터셋의 키 값을 가져옵니다. 파라미터셋 객체를 반환합니다."""
    pset_key, description = _action_info.get(action_key, None)
    if not pset_key:
        return None
    return getattr(app.api.HParameterSet, f"H{pset_key}")

In [ ]:
#| export
@patch
def get_charshape(app: App):
    action = app.actions.CharShape()
    p = action.pset
    return CharShape(p)

In [ ]:
app.get_charshape()

<CharShape: hangul_font=함초롬바탕, latin_font=함초롬바탕, text_color=0, fontsize=10.0, bold=0, italic=0, superscript=0, subscript=0, offset=0, spacing=0, ratio=100, shade_color=#ffffff, shadow_color=#c0c0c0, shadow_offset_x=10, shadow_offset_y=10, strike_out_type=0, strike_out_color=0, underline_type=0, underline_shape=0, underline_color=0, outline_type=0>

In [ ]:
#| export
@patch
def set_charshape(app: App, charshape: CharShape):
    """`현재 위치의 글자 모양을 조정합니다."""
    action = app.actions.CharShape()
    set_pset(action.pset, charshape.todict())
    return action.run()

In [ ]:
charshape = CharShape()
charshape.font = "바탕체"
app.set_charshape(charshape)

True

In [ ]:
app.get_charshape()

<CharShape: hangul_font=바탕체, latin_font=바탕체, text_color=0, fontsize=10.0, bold=0, italic=0, superscript=0, subscript=0, offset=0, spacing=0, ratio=100, shade_color=#ffffff, shadow_color=#c0c0c0, shadow_offset_x=10, shadow_offset_y=10, strike_out_type=0, strike_out_color=0, underline_type=0, underline_shape=0, underline_color=0, outline_type=0>

In [ ]:
#| export
@patch
def get_parashape(app: App):
    action = app.actions.ParagraphShape()
    p = action.pset

    return get_parashape_pset(p)

In [ ]:
app.get_parashape()

{'AlignType': 0,
 'AutoSpaceEAsianEng': 0,
 'AutoSpaceEAsianNum': 0,
 'BorderConnect': 0,
 'BorderOffsetBottom': 0,
 'BorderOffsetLeft': 0,
 'BorderOffsetRight': 0,
 'BorderOffsetTop': 0,
 'BorderText': 0,
 'BreakLatinWord': 0,
 'BreakNonLatinWord': 1,
 'Checked': 0,
 'Condense': 0,
 'FontLineHeight': 0,
 'HeadingType': 0,
 'Indentation': 0,
 'KeepLinesTogether': 0,
 'KeepWithNext': 0,
 'LeftMargin': 0,
 'Level': 0,
 'LineSpacing': 160,
 'LineSpacingType': 0,
 'LineWrap': 0,
 'NextSpacing': 0,
 'PagebreakBefore': 0,
 'PrevSpacing': 0,
 'RightMargin': 0,
 'SnapToGrid': 1,
 'SuppressLineNum': 0,
 'TailType': 0,
 'TextAlignment': 0,
 'WidowOrphan': 0}

In [ ]:
#| export


@patch
def set_parashape(app: App, parashape: ParaShape):
    action = app.actions.ParagraphShape()
    set_pset(action.pset, parashape.todict())
    return action.run()

In [ ]:
app.set_parashape(ParaShape(line_spacing=200))

True

In [ ]:
#| export
@patch
def insert_text(
    app: App,
    text: str,
    charshape: CharShape,
):
    """`text를 입력합니다."""
    app.set_charshape(charshape)
    insert_text = app.actions.InsertText()
    p = insert_text.pset
    p.Text = text
    insert_text.run()
    return

In [ ]:
app.insert_text("안녕하세요", charshape)

In [ ]:
app.set_parashape(ParaShape(right_margin=20, align_type="Left"))

True

In [ ]:
parashape = app.actions.ParagraphShape()

In [ ]:
parashape.pset.RightMargin

5660

In [ ]:
#| export
mask_options = {
    "Normal": 0x00,  # "본문을 대상으로 검색한다.(서브리스트를 검색하지 않는다.)"
    "Char": 0x01,  # "char 타입 컨트롤 마스크를 대상으로 한다.(강제줄나눔, 문단 끝, 하이픈, 묶움빈칸, 고정폭빈칸, 등...)"
    "Inline": 0x02,  # "inline 타입 컨트롤 마스크를 대상으로 한다.(누름틀 필드 끝, 등...)"
    "Ctrl": 0x04,  # "extende 타입 컨트롤 마스크를 대상으로 한다.(바탕쪽, 프레젠테이션, 다단, 누름틀 필드 시작, Shape Object, 머리말, 꼬리말, 각주, 미주, 번호관련 컨트롤, 새 번호 관련 컨트롤, 감추기, 찾아보기, 글자 겹침, 등...)"
    "All": None,
}

scan_spos_keys = {
    "Current": 0x0000,  # "캐럿 위치부터. (시작 위치)",
    "Specified": 0x0010,  # "특정 위치부터. (시작 위치)",
    "Line": 0x0020,  # "줄의 시작부터. (시작 위치)",
    "Paragraph": 0x0030,  # "문단의 시작부터. (시작 위치)"
    "Section": 0x0040,  # "구역의 시작부터. (시작 위치)"
    "List": 0x0050,  # "리스트의 시작부터. (시작 위치)"
    "Control": 0x0060,  # "컨트롤의 시작부터. (시작 위치)"
    "Document": 0x0070,  # "문서의 시작부터. (시작 위치)"
}

scan_epos_keys = {
    "Current": 0x0000,  # "캐럿 위치까지. (끝 위치)"
    "Specified": 0x0001,  # "특정 위치까지. (끝 위치)"
    "Line": 0x0002,  # "줄의 끝까지. (끝 위치)"
    "Paragraph": 0x0003,  # "문단의 끝까지. (끝 위치)"
    "Section": 0x0004,  # "구역의 끝까지. (끝 위치)"
    "List": 0x0005,  # "리스트의 끝까지. (끝 위치)"
    "Control": 0x0006,  # "컨트롤의 끝까지. (끝 위치)"
    "Document": 0x0007,  # "문서의 끝까지. (끝 위치)"
}

scan_directions = {
    "Forward": 0x0000,  # "정뱡향. (검색 방향)"
    "Backward": 0x0100,  # "역방향. (검색 방향)"
}


def _get_text(app):
    """스캔한 텍스트 텍스트 제너레이터"""
    flag, text = 2, ""
    while flag not in [0, 1, 101, 102]:
        flag, text = app.api.GetText()
        yield text


@patch
@contextmanager
def scan(
    app: App,
    option="All",
    selection=False,
    scan_spos="Document",
    scan_epos="Document",
    spara=None,
    spos=None,
    epara=None,
    epos=None,
    scan_direction="Forward",
):
    # set start and end position
    spos_id = get_value(scan_spos_keys, scan_spos)
    epos_id = get_value(scan_epos_keys, scan_epos)
    range_ = spos_id + epos_id
    # if selection
    if selection:
        range_ = 0x00FF  # "검색의 범위를 블록으로 제한."

    # set direction
    direction = get_value(scan_directions, scan_direction)
    range_ = range_ + direction
    app.api.InitScan(
        option=get_value(mask_options, option),
        Range=range_,
        spara=spara,
        spos=spos,
        epara=epara,
        epos=epos,
    )
    yield _get_text(app)
    app.api.ReleaseScan()

In [ ]:
#| export


def move_to_line(app: App, text):
    """인자로 전달한 텍스트가 있는 줄의 시작지점으로 이동합니다."""
    with app.scan(scan_spos="Line") as scan:
        for line in scan:
            if text in line:
                return app.move(key="ScanPos")
    return False

In [ ]:
#| export
move_ids = {
    "Main": 0,  # 루트 리스트의 특정 위치.(para pos로 위치 지정)
    "CurList": 1,  # 현재 리스트의 특정 위치.(para pos로 위치 지정)
    "TopOfFile": 2,  # 문서의 시작으로 이동.
    "BottomOfFile": 3,  # 문서의 끝으로 이동.
    "TopOfList": 4,  # 현재 리스트의 시작으로 이동
    "BottomOfList": 5,  # 현재 리스트의 끝으로 이동
    "StartOfPara": 6,  # 현재 위치한 문단의 시작으로 이동
    "EndOfPara": 7,  # 현재 위치한 문단의 끝으로 이동
    "StartOfWord": 8,  # 현재 위치한 단어의 시작으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "EndOfWord": 9,  # 현재 위치한 단어의 끝으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "NextPara": 10,  # 다음 문단의 시작으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "PrevPara": 11,  # 앞 문단의 끝으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "NextPos": 12,  # 한 글자 뒤로 이동.(서브 리스트를 옮겨 다닐 수 있다.)
    "PrevPos": 13,  # 한 글자 앞으로 이동.(서브 리스트를 옮겨 다닐 수 있다.)
    "NextPosEx": 14,  # 한 글자 뒤로 이동.(서브 리스트를 옮겨 다닐 수 있다. 머리말/꼬리말, 각주/미주, 글상자 포함.)
    "PrevPosEx": 15,  # 한 글자 앞으로 이동.(서브 리스트를 옮겨 다닐 수 있다. 머리말/꼬리말, 각주/미주, 글상자 포함.)
    "NextChar": 16,  # 한 글자 뒤로 이동.(현재 리스트만을 대상으로 동작한다.)
    "PrevChar": 17,  # 한 글자 앞으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "NextWord": 18,  # 한 단어 뒤로 이동.(현재 리스트만을 대상으로 동작한다.)
    "PrevWord": 19,  # 한 단어 앞으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "NextLine": 20,  # 한 줄 아래로 이동.
    "PrevLine": 21,  # 한 줄 위로 이동.
    "StartOfLine": 22,  # 현재 위치한 줄의 시작으로 이동.
    "EndOfLine": 23,  # 현재 위치한 줄의 끝으로 이동.
    "ParentList": 24,  # 한 레벨 상위로 이동한다.
    "TopLevelList": 25,  # 탑레벨 리스트로 이동한다.
    "RootList": 26,  # 루트 리스트로 이동한다. 현재 루트 리스트에 위치해 있어 더 이상 상위 리스트가 없을 때는 위치 이동 없이 반환한다. 이동한 후의 위치는 상위 리스트에서 서브리스트가 속한 컨트롤 코드가 위치한 곳이다. 위치 이동시 셀렉션은 무조건 풀린다.
    "CurrentCaret": 27,  # 현재 캐럿이 위치한 곳으로 이동한다. (캐럿 위치가 뷰의 맨 위쪽으로 올라간다. )
    "LeftOfCell": 100,  # 현재 캐럿이 위치한 셀의 왼쪽
    "RightOfCell": 101,  # 현재 캐럿이 위치한 셀의 오른쪽
    "UpOfCell": 102,  # 현재 캐럿이 위치한 셀의 위쪽
    "DownOfCell": 103,  # 현재 캐럿이 위치한 셀의 아래쪽
    "StartOfCell": 104,  # 현재 캐럿이 위치한 셀에서 행(row)의 시작
    "EndOfCell": 105,  # 현재 캐럿이 위치한 셀에서 행(row)의 끝
    "TopOfCell": 106,  # 현재 캐럿이 위치한 셀에서 열(column)의 시작
    "BottomOfCell": 107,  # 현재 캐럿이 위치한 셀에서 열(column)의 끝
    "ScrPos": 200,  # 한/글 문서장에서의 screen 좌표로서 위치를 설정 한다.
    "ScanPos": 201,  # GetText() 실행 후 위치로 이동한다.
}


@patch
def move(app: App, key="ScanPos", para=None, pos=None):
    """키워드를 바탕으로 캐럿 위치를 이동시킵니다."""

    move_id = get_value(move_ids, key)
    return app.api.MovePos(moveID=move_id, Para=para, pos=pos)

In [ ]:
get_value(move_ids, "ScanPos")

201

In [ ]:
@patch
def setup_page(
    app: App,  # app
    top=20,  # 윗 여백
    bottom=10,  # 아래 여백
    right=20,  # 오른쪽 여백
    left=20,  # 왼쪽 여백
    header=15,  # 머릿말
    footer=5,  # 꼬릿말
    gutter=0,
):  # 제본
    """
    페이지를 설정합니다.
    """
    action = app.actions.PageSetup()
    p = action.pset

    p.PageDef.TopMargin = app.api.MiliToHwpUnit(top)
    p.PageDef.HeaderLen = app.api.MiliToHwpUnit(header)
    p.PageDef.RightMargin = app.api.MiliToHwpUnit(right)
    p.PageDef.BottomMargin = app.api.MiliToHwpUnit(bottom)
    p.PageDef.FooterLen = app.api.MiliToHwpUnit(footer)
    p.PageDef.LeftMargin = app.api.MiliToHwpUnit(left)
    p.PageDef.GutterLen = app.api.MiliToHwpUnit(gutter)

    return action.run()

In [ ]:
#| export
size_options = {
    "realSize": 0,  # 이미지를 원래의 크기로 삽입한다.
    "specificSize": 1,  # width와 height에 지정한 크기로 그림을 삽입한다.
    "cellSize": 2,  # 현재 캐럿이 표의 셀안에 있을 경우, 셀의 크기에 맞게 자동 조정하여 삽입한다.
    "cellSizeWithSameRatio": 3,  # 현재 캐럿이 표의 셀 안에 있을 경우, 셀의 크기에 맞추어 원본 이미지의 가로 세로 비율이 동일하게 확대/축소하여 삽입한다.
}

effects = {
    "RealPicture": 0,  # 원본
    "GrayScale": 1,  # 그레이 스케일
    "BlackWhite": 2,  # 흑백효과
}


@patch
def insert_picture(
    app: App,  # app
    fpath,  # 그림 위치
    width=None,  # 넓이
    height=None,  # 높이
    size_option="realSize",  # 사이즈 옵션
    reverse=False,  # 이미지 반전 여부
    watermark=False,  # 워커마크 여부
    effect="RealPicture",  # 화면 효과
):
    """
    사이즈를 지정하여 사진 삽입
    """
    path = Path(fpath)
    sizeoption = get_value(size_options, size_option)
    effect = get_value(effects, effect)
    return app.api.InsertPicture(
        path.absolute().as_posix(),
        Width=width,
        Height=height,
        sizeoption=sizeoption,
        reverse=reverse,
        watermark=watermark,
        effect=effect,
    )

In [ ]:
#| export


@patch
def select_text(app: App, option: str = "Line"):
    """
    한줄을 선택합니다.
    """
    select_options = {
        "Doc": (app.actions.MoveDocBegin, app.actions.MoveSelDocEnd),
        "Para": (app.actions.MoveParaBegin, app.actions.MoveSelParaEnd),
        "Line": (app.actions.MoveLineBegin, app.actions.MoveSelLineEnd),
        "Word": (app.actions.MoveWordBegin, app.actions.MoveSelWordEnd),
    }
    begin, end = select_options.get(option)
    return begin().run(), end().run()

In [ ]:
app.select_text()

(True, True)

In [ ]:
app.set_charshape(CharShape(font="바탕", fontsize=20))

True

In [ ]:
#| export
@patch
def get_selected_text(app: App):
    """
    선택된 영역의 텍스트를 불러온다.
    """
    with app.scan(selection=True) as scan:
        text = "\n".join(scan)
    return text

In [ ]:
app.get_selected_text()

'안녕하세요\n'

In [ ]:
#| export
@patch
def get_text(app: App, spos="Line", epos="Line"):
    """
    텍스트를 가져옵니다. 기본은 현재 문장입니다.
    """
    with app.scan(scan_spos=spos, scan_epos=epos) as txts:
        text = "".join(txts)
    return text

In [ ]:
app.get_text()

'안녕하세요\r\n'

In [ ]:
#| export
directions = {"Forward": 0, "Backward": 1, "All": 2}

In [ ]:
#| export
@patch
def find_text(
    app: App,
    text="",
    text_fontcolor=None,  # 찾을 폰트 색
    fontsize=None,  # 찾을 폰트 크기(height)
    fontname="",  # 찾을 글꼴
    fonttype=1,  # 찾을 글꼴 타입 TTF = 1, HTF = 2
    fontratio=None,  # 찾을 장평
    text_color=None,
    spacing=None,  # 찾을 자간
    bold=None,  # 찾을 볼드
    italic=None,  # 찾을 이텔릭
    underline=None,  # 찾을 밑줄
    strike_out=None,  # 찾을 취소선
    ignore_message=True,  # 메시지 무시 여부
    direction="Forward",  # 찾을 방향
    match_case=False,  # 대소문자 구분
    all_word_forms=False,  # 문자열 결합
    several_words=False,  #  여러 단어 찾기
    use_wild_cards=False,  # 아무개 문자
    whole_word_only=False,  # 온전한 낱말
    replace_mode=False,  # 찾아 바꾸기 모드
    ignore_find_string=False,  # 찾을 문자열 무시
    ignore_replace_string=False,  # 바꿀 문자열 무시
    find_style="",  # 찾을 스타일
    replace_style="",
    find_jaso=False,  # 자소로 찾기
    find_reg_exp=False,  # 정규표현식으로 찾기
    find_type=False,  # 다시 찾기를 할 때 마지막으로 실한 찾기를 할 경우 True, 찾아가기를 할경우 False
):
    """
    text를 찾습니다.
    찾지못하면 False를 반환합니다.
    """

    action = app.actions.RepeatFind()
    p = action.pset

    # set options
    p.FindString = text
    p.IgnoreMessage = ignore_message
    p.MatchCase = match_case
    p.AllWordForms = all_word_forms
    p.Direction = get_value(directions, direction)
    p.SeveralWords = several_words
    p.UseWildCards = use_wild_cards
    p.WholeWordOnly = whole_word_only
    p.ReplaceMode = replace_mode
    p.IgnoreFindString = ignore_find_string
    p.IgnoreReplaceString = ignore_replace_string
    p.FindStyle = find_style
    p.ReplaceStyle = replace_style
    p.FindJaso = find_jaso
    p.FindRegExp = find_reg_exp
    p.FindType = find_type

    # set old charshape
    set_charshape_pset(
        p.FindCharShape,
        face_name=fontname,
        font_type=fonttype,
        text_color=text_color,
        bold=bold,
        italic=italic,
        strike_out_type=strike_out,
        underline_type=underline,
        ratio=fontratio,
        spacing=spacing,
    )

    return action.run()

In [ ]:
app.find_text("안녕하세요")

TypeError: set_charshape_pset() got an unexpected keyword argument 'face_name'

In [ ]:
#| export
@patch
def replace_all(
    app: App,
    old="",
    new="",
    old_text_color=None,  # 찾을 폰트 색
    new_text_color=None,  # 바꿀 폰트 색
    old_fontsize=None,  # 찾을 폰트 크기(height)
    new_fontsize=None,  # 바꿀 폰트 크기(height)
    old_fontname="",  # 찾을 글꼴
    old_fonttype=1,  # 찾을 글꼴 타입 TTF = 1, HTF = 2
    new_fontname="",  # 바꿀 글꼴
    new_fonttype=1,  # 바꿀 글꼴 타입 TTF = 1, HTF = 2
    old_fontratio=None,  # 찾을 장평
    new_fontratio=None,  # 바꿀 장평
    old_spacing=None,  # 찾을 자간
    new_spacing=None,  # 바꿀 자간
    old_bold=None,  # 찾을 볼드
    new_bold=None,  # 바꿀 볼드
    old_italic=None,  # 찾을 이텔릭
    new_italic=None,  # 바꿀 이텔릭
    old_underline=None,  # 찾을 밑줄
    new_underline=None,  # 바꿀 밑줄
    old_strike_out=None,  # 찾을 취소선
    new_strike_out=None,  # 바꿀 취소선
    ignore_message=True,  # 메시지 무시 여부
    direction="All",  # 찾을 방향
    match_case=False,  # 대소문자 구분
    all_word_forms=False,  # 문자열 결합
    several_words=False,  #  여러 단어 찾기
    use_wild_cards=False,  # 아무개 문자
    whole_word_only=False,  # 온전한 낱말
    auto_spell=True,  # 토시 자동 교정
    replace_mode=True,  # 찾아 바꾸기 모드
    ignore_find_string=False,  # 찾을 문자열 무시
    ignore_replace_string=False,  # 바꿀 문자열 무시
    find_style="",  # 찾을 스타일
    replace_style="",  # 바꿀 스타일
    find_jaso=False,  # 자소로 찾기
    find_reg_exp=False,  # 정규표현식으로 찾기
    find_type=True,  # 다시 찾기를 할 때 마지막으로 실한 찾기를 할 경우 True, 찾아가기를 할경우 False
):
    action = app.actions.AllReplace()
    p = action.pset

    # set options
    p.FindString = old
    p.ReplaceString = new
    p.IgnoreMessage = ignore_message
    p.MatchCase = match_case
    p.AllWordForms = all_word_forms
    p.Direction = get_value(directions, direction)
    p.SeveralWords = several_words
    p.UseWildCards = use_wild_cards
    p.WholeWordOnly = whole_word_only
    p.AutoSpell = auto_spell
    p.ReplaceMode = replace_mode
    p.IgnoreFindString = ignore_find_string
    p.IgnoreReplaceString = ignore_replace_string
    p.FindStyle = find_style
    p.ReplaceStyle = replace_style
    p.FindJaso = find_jaso
    p.FindRegExp = find_reg_exp
    p.FindType = find_type

    # set old charshape
    old_charshape = {
        "FaceNameHangul": old_fontname,
        "TextColor": old_text_color,
    }

    set_charshape_pset(
        p.FindCharShape,
        face_name=old_fontname,
        text_color=old_text_color,
        font_type=old_fonttype,
        bold=old_bold,
        italic=old_italic,
        strike_out_type=old_strike_out,
        underline_type=old_underline,
        ratio=old_fontratio,
        spacing=old_spacing,
    )
    set_charshape_pset(
        p.ReplaceCharShape,
        face_name=new_fontname,
        text_color=new_text_color,
        font_type=new_fonttype,
        bold=new_bold,
        italic=new_italic,
        strike_out_type=new_strike_out,
        underline_type=new_underline,
        ratio=new_fontratio,
        spacing=new_spacing,
    )

    return action.run()

In [ ]:
app.replace_all("t", "txt", new_fontname="가는안상수체", new_fontsize=21, new_fonttype=1)

In [ ]:
app.replace_all(
    "",
    "",
    old_fontname="바탕체",
    new_fontname="바탕체",
    ignore_find_string=True,
    ignore_replace_string=True,
    new_fontsize=11,
    new_fonttype=1,
)

In [ ]:
#| export
@patch
def insert_file(
    app: App,
    fpath,
    keep_charshape=False,
    keep_parashape=False,
    keep_section=False,
    keep_style=False,
):
    """
    파일 끼워 넣기
    """

    action = app.actions.InsertFile()
    p = action.pset
    p.filename = Path(fpath).absolute().as_posix()
    p.KeepCharshape = keep_charshape
    p.KeepParashape = keep_parashape
    p.KeepSection = keep_section
    p.KeepStyle = keep_style

    return action.run()

In [ ]:
#| export


@patch
def set_cell_border(
    app: App,
    top=None,
    right=None,
    left=None,
    bottom=None,
    horizontal=None,
    vertical=None,
    top_width=None,
    right_width=None,
    left_width=None,
    bottom_width=None,
    horizontal_width=None,
    vertical_width=None,
    top_color=None,
    bottom_color=None,
    left_color=None,
    right_color=None,
    horizontal_color=None,
    vertical_color=None,
):
    attrs = {
        "BorderTypeTop": top,
        "BorderTypeRight": right,
        "BorderTypeLeft": left,
        "BorderTypeBottom": bottom,
        "TypeHorz": horizontal,
        "TypeVert": vertical,
        "BorderWidthTop": app.api.HwpLineWidth(f"{top_width}mm") if top_width else None,
        "BorderWidthRight": app.api.HwpLineWidth(f"{right_width}mm")
        if right_width
        else None,
        "BorderWidthLeft": app.api.HwpLineWidth(f"{left_width}mm")
        if left_width
        else None,
        "BorderWidthBottom": app.api.HwpLineWidth(f"{bottom_width}mm")
        if bottom_width
        else None,
        "WidthHorz": app.api.HwpLineWidth(f"{horizontal_width}mm")
        if horizontal_width
        else None,
        "WidthVert": app.api.HwpLineWidth(f"{vertical_width}mm")
        if vertical_width
        else None,
        "BorderColorTop": app.api.RGBColor(get_rgb_tuple(top_color))
        if top_color
        else None,
        "BorderColorRight": app.api.RGBColor(get_rgb_tuple(right_color))
        if right_color
        else None,
        "BorderColorLeft": app.api.RGBColor(get_rgb_tuple(left_color))
        if left_color
        else None,
        "BorderColorBottom": app.api.RGBColor(get_rgb_tuple(bottom_color))
        if bottom_color
        else None,
        "ColorHorz": app.api.RGBColor(get_rgb_tuple(horizontal_color))
        if horizontal_color
        else None,
        "ColorVert": app.api.RGBColor(get_rgb_tuple(vertical_color))
        if vertical_color
        else None,
    }

    action = app.actions.CellFill()
    p = action.pset

    for key, value in attrs.items():
        if value is None:
            continue
        setattr(p, key, value)

    return action.run()

In [ ]:
#| export
@patch
def set_cell_color(
    app: App, bg_color=None, hatch_color="#000000", hatch_style=6, alpha=None
):
    fill_type = windows_brush = None
    if bg_color:
        fill_type = 1
        windows_brush = 1

    attrs = {
        "type": fill_type,
        "WindowsBrush": windows_brush,
        "WinBrushFaceColor": app.api.RGBColor(*get_rgb_tuple(bg_color))
        if bg_color
        else None,
        "WinBrushAlpha": alpha,
        "WinBrushHatchColor": app.api.RGBColor(*get_rgb_tuple(hatch_color))
        if hatch_color
        else None,
        "WinBrushFaceStyle": hatch_style,
    }

    action = app.actions.CellBorderFill()
    p = action.pset

    for key, value in attrs.items():
        if value == None:
            continue
        setattr(p.FillAttr, key, value)

    return action.run()

In [ ]:
app = App()

In [ ]:
app.set_charshape(fontname="문화바탕", font_type=2)

In [ ]:
action = app.actions.CharShape()

In [ ]:
action.pset.FaceNameHangul = "문화돋움"
action.run()

In [ ]:
app.get_charshape()

In [ ]:
app.set_cell_border(bottom=3)

In [ ]:
number = app.api.RGBColor(0, 255, 255)

In [ ]:
action.pset

In [ ]:
int(f"{number:06x}", 16)

In [ ]:
hex(number)

In [ ]:
app.set_cell_color(bg_color=(123, 123, 123), hatch_style=6)

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()